In [1]:
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL.Image as img
import os
from scipy.io import savemat,loadmat

2023-12-20 17:02:13.275264: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 17:02:14.166596: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model=tf.keras.models.load_model("/home/featurize/work/System/F1_M_88p")

2023-12-20 17:07:24.378963: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 17:07:24.384892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 17:07:24.385169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
@tf.function
def test_step(image,label):
    predict=model(image,training=False)
    loss=loss_obj(label,predict)
    
    test_loss(loss)
    test_accuracy(label,predict)

In [4]:
def gettensor(folder_path,file_name):
    im=img.open(os.path.join(folder_path,file_name)).convert('L')
    im=im.resize((1100,1100),img.LANCZOS)
    tens=tf.constant(im,dtype=tf.float32)
    tens=tf.reshape(tens,(1,1100,1100,1))/tf.reduce_max(tens)
    del im
    return tens

In [5]:
test_folder="/home/featurize/data/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test"
test_name=pd.read_csv("/home/featurize/work/data/test_shuffled.csv")["X_ray_image_name"]
test_name=np.array(test_name)

In [6]:
l_test_y=pd.read_csv("/home/featurize/work/data/test_shuffled.csv")["Label"]

def numberize(l_y):
    y=[]
    for lab in l_y:
        if lab=="Normal":
            y.append(0)
        else:
            y.append(1)
    return y

test_y=tf.constant(numberize(l_test_y),dtype=tf.int32)

In [8]:
loss_obj=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer=tf.keras.optimizers.RMSprop(0.001,0.9)

In [9]:
train_loss=tf.keras.metrics.Mean(name="train_loss")
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss=tf.keras.metrics.Mean(name="test_loss")
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [10]:
for i in range(len(test_y)):
            image=gettensor(test_folder,test_name[i])
            label=test_y[i]
            test_step(image,label)
            
print(f"Accuracy: %{test_accuracy.result()*100}")
print(f"length:{len(test_y)}")

/tmp/ipykernel_19074/3197959897.py:3: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im=im.resize((1100,1100),img.LANCZOS)
2023-12-20 17:13:25.855826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-20 17:13:26.636061: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Accuracy: %90.224365234375
length:624
